In [10]:
!pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd
from tqdm import tqdm
df = pd.read_csv("F:\Fourth year\Research\My_try\preprocess\process/testingdata.csv")
df.head(20)


<>:3: SyntaxWarning: invalid escape sequence '\F'
<>:3: SyntaxWarning: invalid escape sequence '\F'
C:\Users\DELL\AppData\Local\Temp\ipykernel_8800\1497767738.py:3: SyntaxWarning: invalid escape sequence '\F'
  df = pd.read_csv("F:\Fourth year\Research\My_try\preprocess\process/testingdata.csv")


,Brand,Model,price,Edition,Cleaned_Description
0,Xiaomi,Redmi A2 plus,"Rs 20,000",Xiaomi,full set with box condition 100% check karala ...
1,Xiaomi,Poco M3 Pro,"Rs 20,000",6GB 128GB,small glass creck location kegallemsg witharak...
2,Xiaomi,Redmi Note 9,"Rs 20,000",Xiaomi Redmi Note 9,4+2 gb 6gb ram 128gb romused nothing to any er...
3,Xiaomi,Redmi Note 12,"Rs 20,000",8GB 256GB,redmi nite 1212 gb ram128 romrs.25000
4,Xiaomi,12C,"Rs 20,000",Redmi 12C,financial issue good condition
5,Xiaomi,Redmi A2 plus,"Rs 20,000",NaN,3gb/64gbphone eke kisi awlak na full condition...
6,Xiaomi,Redmi Note 10,"Rs 20,000",NaN,redmi note 10 side fingerprint face small gras...
7,Xiaomi,Poco M4 Pro 5G,"Rs 20,000",POCO M4 Pro (8/128),"# ( ){8+3/128 ~ } : , .~ condition, genuine un..."
8,Xiaomi,Redmi 9t,"Rs 20,000",Xiaomi,kisima aulak nh salli hadisiyak nisa wikunane
9,Xiaomi,Redmi 9,"Rs 20,000",NaN,mudal hadisiyak sadaha wikinimata


In [12]:
df.shape

(1922, 5)

In [13]:
df.columns = df.columns.str.strip().str.replace(":", "").str.replace(" ", "_")

In [14]:
import openai

openai.api_key = "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"
openai.base_url = "https://api.groq.com/openai/v1"

client = openai.OpenAI(
    api_key=openai.api_key,
    base_url=openai.base_url,
)



In [15]:
import pandas as pd
from collections import defaultdict
spec_df = pd.read_excel("F:\Fourth year\Research\My_try\preprocess/phone_specs.xlsx")  # or read_csv
# spec_df["Brand"] = spec_df["Brand"].str.strip()
# spec_df["Model"] = spec_df["Model"].str.strip()
# spec_lookup = defaultdict(list)
# for _, row in spec_df.iterrows():
#     key = (row["Brand"], row["Model"])
#     spec_lookup[key].append(row.to_dict())

spec_df.shape

<>:3: SyntaxWarning: invalid escape sequence '\F'
<>:3: SyntaxWarning: invalid escape sequence '\F'
C:\Users\DELL\AppData\Local\Temp\ipykernel_8800\3770823137.py:3: SyntaxWarning: invalid escape sequence '\F'
  spec_df = pd.read_excel("F:\Fourth year\Research\My_try\preprocess/phone_specs.xlsx")  # or read_csv


(274, 20)

In [16]:
import unicodedata
import re
def clean_text(val):
    if pd.isna(val):
        return ""
    val = str(val).lower().strip()
    val = re.sub(r'\s+', ' ', val)  # replace tabs, multiple spaces, etc. with single space
    val = unicodedata.normalize("NFKD", val)  # remove weird unicode characters
    return val

for col in ["Brand", "Model", "RAM", "Storage"]:
    spec_df[col] = spec_df[col].map(clean_text)

# Clean the search values too

def extract_ram_storage_from_edition(edition):
    if pd.isna(edition):
        return "", ""
    
    text = edition.lower()
    ram_match = re.search(r'(\d+)\s*(gb)?\s*(ram)?', text)
    storage_match = re.search(r'(\d+)\s*(gb)?\s*(rom|storage)?', text)

    ram = ram_match.group(1) if ram_match else ""
    storage = storage_match.group(1) if storage_match else ""
    
    return ram.strip(), storage.strip()

# Usage
edition = "6|128"
ram, storage = extract_ram_storage_from_edition(edition)

def try_match(spec_df, brand, model, ram, storage):
    conditions = [
        # Try full match
        (spec_df["Brand"] == brand) & (spec_df["Model"] == model) &
        (spec_df["RAM"] == ram) & (spec_df["Storage"] == storage),
        
        # Try RAM only
        (spec_df["Brand"] == brand) & (spec_df["Model"] == model) &
        (spec_df["RAM"] == ram),

        # Try Storage only
        (spec_df["Brand"] == brand) & (spec_df["Model"] == model) &
        (spec_df["Storage"] == storage),

        # Try Brand + Model only
        (spec_df["Brand"] == brand) & (spec_df["Model"] == model)
    ]

    for cond in conditions:
        match = spec_df[cond]
        if not match.empty:
            return match.iloc[0].to_dict()
    
    return None



In [17]:
def clean_for_json(d):
    return {
        k: (v.replace('\xa0', ' ').strip() if isinstance(v, str) else v)
        for k, v in d.items()
    }

In [18]:
import json
import re



# Replace these with the single row's values you want to test
brand = clean_text("Xiaomi")
model = clean_text("Redmi 10A")
edition = "2GB/ 32GB"
ram, storage = extract_ram_storage_from_edition(edition)
price = "Rs 17,000"
description = """
display small back glass track used 3 months,orifinal box and charger included

"""
spec_dict = try_match(spec_df, brand, model, ram, storage)

if spec_dict:
    print("✅ Match found:", spec_dict)
else:
    print("❌ No matching spec found.")


prompt = f"""
You are a phone expert. Given a second-hand phone listing and the seller's description, extract detailed phone condition in structured format.

Listing Info:
- Brand: {brand}
- Model: {model}
- Edition: {edition}
- Price: {price}
     {json.dumps(clean_for_json(spec_dict), ensure_ascii=False)}
- Ad Description: \"\"\"{description}\"\"\"

Using this info, fill the following JSON with accurate, inferred, or clearly mentioned values.

Return only JSON with the following fields:
{{

  "RAM": "...",
  "Storage": "...",
  "Battery_Capacity": "...",
  "Front_Camera": "...",
  "Main_Camera": "...",
  "Cam_02": "...",
  "Cam_03": "...",
  "Cam_04": "...",
  "Processor": "...",
  "Weight_Grams":"...",
  "OS_Version": "...",
  "Display_Type": "...",
  "Network":"...",
  "Display_RefreshRate":"...",
  "Has_Face_ID": "...",
  "Has_Fingerprint": "...",  
  "Release_Year": "...",

 
  "Display_Condition": "...",
  "Physical_Condition": "...",
  "Phone_Overheating": "...",
  "Touchscreen_Working": "...",
  "Buttons_Working": "...",
  "Charging_Port_Working": "...",
  "Camera_Working": "...",
  "Fingerprint_Working": "...",
  "Face_ID_Working": "...",
  "Speaker_Clear": "...",
  "Charger_Included: "...",
  "Box_Included: "...",
  "Display_Protection": "...",
  "Original_Accessories": "...",
  "Software_Warranty": "...",
  "Hardware_Warranty": "...",
  "Parts_Changed": "...",

}}
⚠️ Guidelines:
- Use full descriptive sentences where appropriate.
- Do not limit yourself to fixed options like 'Yes/No'.
- Think like a human buyer reading the ad.
"""

# Send prompt to model
try:
    res = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "You extract second-hand smartphone features for price prediction."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
    )
    content = res.choices[0].message.content
    match = re.search(r"\{.*\}", content, re.DOTALL)

    if match:
        json_block = match.group(0)
        json_block = json_block.replace("“", "\"").replace("”", "\"")  # clean smart quotes
        json_block = json_block.replace("‘", "'").replace("’", "'")
        parsed = json.loads(json_block)
        parsed.update({
            "Brand": brand,
            "Model": model,
            "Price": price
        })
       
        print("✅ Extracted JSON:\n", json.dumps(parsed, indent=2))
    else:
        print("⚠️ No JSON found.\nRaw Output:\n", content)

except Exception as e:
    print("❌ Error:", e)



✅ Match found: {'Brand': 'xiaomi', 'Model': 'redmi 10a', 'RAM': '2', 'Storage': '32', 'Battery_Capacity': 5000, 'Front_Camera': 5, 'Main_Camera': 13, 'cam_02': 0, 'cam_03': 0, 'cam_04': 0.0, 'Processor': 'Mediatek Helio G25 (12 nm)', 'weight(gram)': 194.0, 'operating_system': 'Android 11, MIUI 12.5', 'Display_Type': 'IPS LCD', 'Network': '4G', 'DisplayRefreshRate': 60, 'face_id': 'yes', 'finger_print': 'yes', 'new_price': nan, 'release_year': 2022.0}
✅ Extracted JSON:
 {
  "RAM": "2GB",
  "Storage": "32GB",
  "Battery_Capacity": 5000,
  "Front_Camera": 5,
  "Main_Camera": 13,
  "Cam_02": 0,
  "Cam_03": 0,
  "Cam_04": 0.0,
  "Processor": "Mediatek Helio G25 (12 nm)",
  "Weight_Grams": 194.0,
  "OS_Version": "Android 11, MIUI 12.5",
  "Display_Type": "IPS LCD",
  "Network": "4G",
  "Display_RefreshRate": 60,
  "Has_Face_ID": "yes",
  "Has_Fingerprint": "yes",
  "Release_Year": 2022.0,
  "Display_Condition": "Small display scratches",
  "Physical_Condition": "Used, 3 months old",
  "Phone

In [ ]:
import json
import re
import os
import pandas as pd
from tqdm import tqdm

# Assume helper functions are already defined: clean_text, extract_ram_storage_from_edition, try_match, clean_for_json

input_file = "F:\Fourth year\Research\My_try\preprocess\process/testingdata.csv"
df = pd.read_csv(input_file)

output_dir = "grok_outputs"
os.makedirs(output_dir, exist_ok=True)

batch_size = 500
num_batches = (len(df) + batch_size - 1) // batch_size  # ceiling division

for batch_idx in range(num_batches):
    start = batch_idx * batch_size
    end = min(start + batch_size, len(df))
    df_batch = df.iloc[start:end]

    outputs = []

    for i, row in tqdm(df_batch.iterrows(), total=len(df_batch), desc=f"Batch {batch_idx + 1}/{num_batches}"):
        try:
            brand = clean_text(row["Brand"])
            model = clean_text(row["Model"])
            edition = row.get("Edition", "") or ""
            ram, storage = extract_ram_storage_from_edition(edition)
            price = row.get("price", "")
            description = row.get("Cleaned_Description", "")
            condition = row.get("Condition", "")
            spec_dict = try_match(spec_df, brand, model, ram, storage)
            spec_json = json.dumps(clean_for_json(spec_dict), ensure_ascii=False) if spec_dict else ""

            prompt = f"""
You are a phone expert. Given a second-hand phone listing and the seller's description, extract detailed phone condition in structured format.

Listing Info:
- Brand: {brand}
- Model: {model}
- Edition: {edition}
- Price: {price}
{spec_json}
- Ad Description: \"\"\"{description}\"\"\"

Using this info, fill the following JSON with accurate, inferred, or clearly mentioned values.

Return only JSON with the following fields:
{{
  
  "RAM": "...",
  "Storage": "...",
  "Battery_Capacity": "...",
  "Front_Camera": "...",
  "Main_Camera": "...",
  "Cam_02": "...",
  "Cam_03": "...",
  "Cam_04": "...",
  "Processor": "...",
  "Weight_Grams":"...",
  "OS_Version": "...",
  "Display_Type": "...",
  "Network":"...",
  "Display_RefreshRate":"...",
  "Has_Face_ID": "...",
  "Has_Fingerprint": "...",  
  "Release_Year": "...",

 
  "Display_Condition": "...",
  "Physical_Condition": "...",
  "Phone_Overheating": "...",
  "Touchscreen_Working": "...",
  "Buttons_Working": "...",
  "Charging_Port_Working": "...",
  "Camera_Working": "...",
  "Fingerprint_Working": "...",
  "Face_ID_Working": "...",
  "Speaker_Clear": "...",
  "Charger_Included: "...",
  "Box_Included: "...",
  "Display_Protection": "...",
  "Original_Accessories": "...",
  "Software_Warranty": "...",
  "Hardware_Warranty": "...",
  "Parts_Changed": "...",
}}

⚠️ Guidelines:
- Use full descriptive sentences where appropriate.
- Do not limit yourself to fixed options like 'Yes/No'.
- Think like a human buyer reading the ad.
"""

            res = client.chat.completions.create(
                model="llama3-8b-8192",
                messages=[
                    {"role": "system", "content": "You extract second-hand smartphone features for price prediction."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
            )

            content = res.choices[0].message.content
            match = re.search(r"\{.*\}", content, re.DOTALL)
            if match:
                json_block = match.group(0)
                json_block = json_block.replace("“", "\"").replace("”", "\"") \
                                       .replace("‘", "'").replace("’", "'")
                parsed = json.loads(json_block)
                parsed.update({
                    "Brand": brand,
                    "Model": model,
                    "Price": price,
                    "Condition":condition
                })
                outputs.append(parsed)
            else:
                outputs.append({"error": "No JSON found", "raw_output": content})

        except Exception as e:
            outputs.append({"error": str(e)})

    # Save batch output
    batch_output_file = f"{output_dir}/grok_output_batch_{batch_idx + 1}.csv"
    pd.DataFrame(outputs).to_csv(batch_output_file, index=False)
    print(f"✅ Saved batch {batch_idx + 1} to {batch_output_file}")

print("🎉 All batches processed.")


<>:9: SyntaxWarning: invalid escape sequence '\F'
<>:9: SyntaxWarning: invalid escape sequence '\F'
C:\Users\DELL\AppData\Local\Temp\ipykernel_8800\4080448298.py:9: SyntaxWarning: invalid escape sequence '\F'
  input_file = "F:\Fourth year\Research\My_try\preprocess\process/testingdata.csv"
Batch 1/4: 100%|██████████| 500/500 [2:14:53<00:00, 16.19s/it]    


✅ Saved batch 1 to grok_outputs/grok_output_batch_1.csv


Batch 2/4:  30%|███       | 152/500 [09:43<36:42,  6.33s/it]    

In [ ]:

# structured_df = pd.DataFrame(outputs)
# structured_df.to_csv("grok_detailed_output.csv", index=False)
# print("✅ Extraction complete. Output saved to 'grok_detailed_output.csv'")